In [2]:
%matplotlib inline

In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

\begin{align}m_{u\to v}^{(l)} = M^{(l)}\left(h_v^{(l-1)}, h_u^{(l-1)}, e_{u\to v}^{(l-1)}\right)\end{align}
#---1
\begin{align}m_{v}^{(l)} = \sum_{u\in\mathcal{N}(v)}m_{u\to v}^{(l)}\end{align}

\begin{align}h_v^{(l)} = U^{(l)}\left(h_v^{(l-1)}, m_v^{(l)}\right)\end{align}

<!-- \begin{align}m_{u\to v}^{(l)} = M^{(l)}\left(h_v^{(l-1)}, h_u^{(l-1)}, e_{u\to v}^{(l-1)}\right)\end{align}

\begin{align}m_{v}^{(l)} = \sum_{u\in\mathcal{N}(v)}m_{u\to v}^{(l)}\end{align}

\begin{align}h_v^{(l)} = U^{(l)}\left(h_v^{(l-1)}, m_v^{(l)}\right)\end{align} -->

\begin{align}h_{\mathcal{N}(v)}^k\leftarrow \text{Average}\{h_u^{k-1},\forall u\in\mathcal{
N}(v)\}\end{align}

\begin{align}h_v^k\leftarrow \text{ReLU}\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)\end{align}


In [4]:
import dgl.function as fn

In [25]:
class SAGEConv(nn.Module):
    def __init__(self,in_feat,out_feat):
        super(SAGEConv,self).__init__()
        self.linear=nn.Linear(in_feat*2,out_feat)
    def forward(self,g,h):
        with g.local_scope():
            g.ndata['h']=h
            g.update_all(message_func=fn.copy_u('h','m'),reduce_func=fn.mean('m','h_N'))
            h_N = g.ndata['h_N']
            h_total=torch.cat([h,h_N],dim=1)
            return self.linear(h_total)

* Message function ``fn.copy_u('h', 'm')`` that
  copies the node feature under name ``'h'`` as *messages* sent to
  neighbors.

* Reduce function ``fn.mean('m', 'h_N')`` that averages
  all the received messages under name ``'m'`` and saves the result as a
  new node feature ``'h_N'``.

* ``update_all`` tells DGL to trigger the
  message and reduce functions for all the nodes and edges.

In [32]:
class Model(nn.Module):
    def __init__(self,in_feat,h_feat,num_classes):
        super(Model,self).__init__()
        self.conv1=SAGEConv(in_feat,h_feat)
        self.conv2=SAGEConv(h_feat,num_classes)
    def forward(self,g,in_feats):
        h=self.conv1(g,in_feats)
        h=F.relu(h)
        out=self.conv2(g,h)
        return out

In [33]:
import dgl.data

In [34]:
dataset=dgl.data.CoraGraphDataset()
g=dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [56]:
def train(g,model):
    optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
    all_logits=[]
    best_val_acc=0
    best_test_acc=0
    features=g.ndata['feat'] # forward
    labels=g.ndata['label']
    train_mask=g.ndata['train_mask']
    val_mask=g.ndata['val_mask']
    test_mask=g.ndata['test_mask']
    for i in range(200):
        logits=model(g,features)
        pred=logits.argmax(1)
        loss=F.cross_entropy(logits[train_mask],labels[train_mask])
        #compute acc
        train_acc=(pred[train_mask]==labels[train_mask]).float().mean()
        val_acc=(pred[val_mask]==labels[val_mask]).float().mean()
        test_acc=(pred[test_mask]==labels[test_mask]).float().mean()
        #save
        if best_val_acc<val_acc:
            best_val_acc=val_acc
            best_test_acc=test_acc
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())
        if i % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                i, loss, val_acc, best_val_acc, test_acc, best_test_acc))
        
        

In [57]:
model1=Model(g.ndata['feat'].shape[1],16,dataset.num_classes)


In [58]:
train(g,model1)

In epoch 0, loss: 1.949, val acc: 0.122 (best 0.122), test acc: 0.130 (best 0.130)
In epoch 5, loss: 1.871, val acc: 0.472 (best 0.472), test acc: 0.465 (best 0.465)
In epoch 10, loss: 1.729, val acc: 0.576 (best 0.588), test acc: 0.531 (best 0.554)
In epoch 15, loss: 1.515, val acc: 0.598 (best 0.598), test acc: 0.566 (best 0.566)
In epoch 20, loss: 1.238, val acc: 0.602 (best 0.602), test acc: 0.585 (best 0.585)
In epoch 25, loss: 0.931, val acc: 0.626 (best 0.626), test acc: 0.605 (best 0.605)
In epoch 30, loss: 0.642, val acc: 0.660 (best 0.660), test acc: 0.639 (best 0.626)
In epoch 35, loss: 0.410, val acc: 0.714 (best 0.714), test acc: 0.692 (best 0.692)
In epoch 40, loss: 0.249, val acc: 0.732 (best 0.732), test acc: 0.731 (best 0.731)
In epoch 45, loss: 0.150, val acc: 0.744 (best 0.744), test acc: 0.741 (best 0.740)
In epoch 50, loss: 0.092, val acc: 0.748 (best 0.748), test acc: 0.744 (best 0.747)
In epoch 55, loss: 0.059, val acc: 0.746 (best 0.750), test acc: 0.746 (best 0

# customization 定制化

In [59]:
class WeightedSAGEConv(nn.Module):
 
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h, w): # w表示边权重
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(message_func=fn.u_mul_e('h', 'w', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [62]:
def u_mul_e_udf(edges):
    return {'m' : edges.src['h'] * edges.data['w']}

def sum_udf(nodes):
    return {'h': nodes.mailbox['m'].sum(1)}

In [60]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges()).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges()).to(g.device))
        return h
    
model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.952, val acc: 0.114 (best 0.114), test acc: 0.103 (best 0.103)
In epoch 5, loss: 1.872, val acc: 0.090 (best 0.132), test acc: 0.098 (best 0.127)
In epoch 10, loss: 1.721, val acc: 0.356 (best 0.356), test acc: 0.359 (best 0.359)
In epoch 15, loss: 1.492, val acc: 0.562 (best 0.562), test acc: 0.574 (best 0.574)
In epoch 20, loss: 1.191, val acc: 0.646 (best 0.646), test acc: 0.658 (best 0.658)
In epoch 25, loss: 0.858, val acc: 0.686 (best 0.686), test acc: 0.706 (best 0.706)
In epoch 30, loss: 0.551, val acc: 0.742 (best 0.742), test acc: 0.737 (best 0.737)
In epoch 35, loss: 0.320, val acc: 0.758 (best 0.762), test acc: 0.753 (best 0.747)
In epoch 40, loss: 0.175, val acc: 0.756 (best 0.762), test acc: 0.754 (best 0.747)
In epoch 45, loss: 0.096, val acc: 0.758 (best 0.762), test acc: 0.764 (best 0.747)
In epoch 50, loss: 0.056, val acc: 0.756 (best 0.762), test acc: 0.762 (best 0.747)
In epoch 55, loss: 0.035, val acc: 0.758 (best 0.762), test acc: 0.762 (best 0